In [21]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from os import walk
import xlwt 
from xlwt import Workbook 
import tensorflow as tf
from tensorflow import keras
from sklearn.cluster import KMeans

In [22]:
from keras.models import load_model
model = load_model("/content/drive/My Drive/Project_MAIN/checkpoint.h5")

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [23]:
model.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=["accuracy"])

In [24]:
def detect_white_roi(image_part,clt,bg_class):
  sayac = 0
  a = 0
  b = 0
  c = 0
  for i in range(0,100,1):
    for j in range(0,100,1):
      point = np.array(image_part[i,j,:]).reshape((1,3))
      result = clt.predict(point)
      if result == bg_class:
        sayac += 1
        a += point[0,0]
        b += point[0,1]
        c += point[0,2]

  if (sayac>600):
    white_bg = np.array([[a],[b],[c]])/sayac
    return (False,white_bg)
  else:
    return (True,[[0],[0],[0]])

In [25]:

def cluster_colors(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    wn_clusters = 4
    point = np.array(image[0,0,:])
    image_part = image[175:275,150:250,:]
    white_fg = True

    ### KMEANS 

    image = image.reshape((image.shape[0] * image.shape[1], 3))

    clt = KMeans(n_clusters = wn_clusters)
    clt.fit(image)

    centroids = clt.cluster_centers_

    ### HISTOGRAM

    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins = numLabels)
    hist = hist.astype("float")

    ### CHECK WHITE ROI

    point = point.reshape((1,3))
    index = clt.predict(point)

    (white_fg,white_bg) = detect_white_roi(image_part,clt,index)
     
    ### DELETE BACKGROUND
    n,m = centroids.shape

    if (white_fg):
      centroids = np.delete(centroids,index,0)
      hist = np.delete(hist,index,0)
    else:
      centroids[index,0] = white_bg[0]
      centroids[index,1] = white_bg[1]
      centroids[index,2] = white_bg[2]


    n,m = centroids.shape

    ### ORDER CENTROIDS BASED ON THEIR %PERCENTS
    for i in range (0,n,1):
      for j in range(0,n,1):
        if (i == j):
          continue
        
        if (hist[i]>hist[j]):
          temp = hist[j]
          hist[j] = hist[i]
          hist[i] = temp

          centroids[[j,i]] = centroids[[i,j]]

    if (n >= 3):
      centroids = centroids[0:2,:]
      hist = hist[0:2]

    hist /= hist.sum()
    
    return (centroids,hist)

In [26]:
def write_img(image,result,i):
  image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
  
  if result == 0:
    address = "/content/drive/My Drive/rtest3saves/gomlek/"+str(i)+".jpg"
  elif result == 1:
    address = "/content/drive/My Drive/rtest3saves/pantolon/"+str(i)+".jpg"
  elif result == 2:
    address = "/content/drive/My Drive/rtest3saves/t-shirt/"+str(i)+".jpg"
  cv2.imwrite(address,image)
  return address

In [27]:
wb = Workbook()
sheet = wb.add_sheet("Rapor",cell_overwrite_ok=True)

def sheet_write(result,centroid,i,address,hist):
  sheet.write(i+1,0,address)
  if result == 0:
    sheet.write(i+1,1, "gomlek")
  elif result == 1:
    sheet.write(i+1,1, "pantolon")
  elif result == 2:
    sheet.write(i+1,1, "t-shirt")
  (n,m) = centroid.shape
  sayac1=0
  for countit in range(2,(3*2+2),3):
    if (n==1 and sayac1==1):
      sheet.write(i+1,countit,"0")
      sheet.write(i+1,countit+1,"0")
      sheet.write(i+1,countit+2,"0")
    else:
      sheet.write(i+1,countit,str(centroid[sayac1,0]))
      sheet.write(i+1,countit+1,str(centroid[sayac1,1]))
      sheet.write(i+1,countit+2,str(centroid[sayac1,2]))
    sayac1 = sayac1 + 1
  
  sayac2=0
  for countit2 in range(3*2+2,7*2+2,4):
    if (n==1 and sayac2==1):
      sheet.write(i+1,countit2,"0")
      sheet.write(i+1,countit2+1,"0")
      sheet.write(i+1,countit2+2,"0")
      sheet.write(i+1,countit2+3,"0")
    else:
     cmyk_codes = list(rgb_to_cmyk(centroid[sayac2,:]/255))
     sheet.write(i+1,countit2,str(cmyk_codes[0]))
     sheet.write(i+1,countit2+1,str(cmyk_codes[1]))
     sheet.write(i+1,countit2+2,str(cmyk_codes[2]))
     sheet.write(i+1,countit2+3,str(cmyk_codes[3]))
    sayac2 = sayac2 + 1
  if (n==1):
    sheet.write(i+1,7*2+2,"1.0")
    sheet.write(i+1,7*2+3,"0.0")
  elif (n>=2):
    sheet.write(i+1,7*2+2,str(hist[0]))
    sheet.write(i+1,7*2+3,str(hist[1]))
  sheet.write(i+1,7*2+4,str(n))

  



In [28]:
def rgb_to_cmyk(centro):
  k = 1 - centro.max()
  c = (1 - centro[0] - k) / (1 - k)
  m = (1 - centro[1] - k) / (1 - k)
  y = (1 - centro[2] - k) / (1 - k)
  return (c,m,y,k)


In [29]:
path_list = []
for (dirpath, dirnames, filenames) in walk("/content/drive/My Drive/rtest3"):
    path_list.extend(filenames)
    break
results = []
for i in range(0,20,1):
  path = dirpath + '/' + path_list[i]
  image = cv2.imread(path)
  centroid,hist = cluster_colors(image)
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  image = cv2.resize(src=image,dsize=(100,125))
  image = image.reshape((1,125,100,3))
  result = model.predict_classes(image)
  image = image.reshape((125,100,3))
  address = write_img(image,result,i)
  sheet_write(result,centroid,i,address,hist)
  results.extend(result)

In [30]:
wb.save("/content/drive/My Drive/Project_MAIN/Reports/rapor13.xls")